In [ ]:
#importing libraries

import pandas as pd
import numpy as np
import random
from numpy import log2 as log
from matplotlib import pyplot as plt
from pprint import pprint
%matplotlib inline
eps = np.finfo(float).eps

In [ ]:
#preprocessing data

df = pd.read_csv('adult.csv', delimiter=', ')

for i in range(10):
    lower = 10*i+1
    upper = 10*(i+1)+1
    replace = 10*i+5
    df['Age'] = df['Age'].replace(to_replace = [x for x in range(lower, upper)], value=replace)

for i in range(10):
    lower = 10000*i+1
    upper = 10000*(i+1)+1
    replace = 10000*i+5000
    df['Capital_gain'] = df['Capital_gain'].replace(to_replace = [x for x in range(lower, upper)], value = replace)

for i in range(10):
    lower = 500*i+1
    upper = 500*(i+1)+1
    replace = 500*i+250
    df['Capital_loss'] = df['Capital_loss'].replace(to_replace = [x for x in range(lower, upper)], value = replace)

for i in range(20):
    lower = 5*i+1
    upper = 5*(i+1)+1
    replace = 5*i
    df['hours_per_week'] = df['hours_per_week'].replace(to_replace = [x for x in range(lower, upper)], value = replace)

for i in range(15):
    lower = 100000*i+1
    upper = 100000*(i+1)+1
    replace = 100000*i+50000
    df['fnlwgt'] = df['fnlwgt'].replace(to_replace = [x for x in range(lower, upper)], value = replace)
    
for key in df.keys():
    values = df[key].value_counts()
    max_key = np.argmax(values)
    df[key] = df[key].replace(to_replace = '?', value = max_key)

In [ ]:
# all functions used

def find_entropy(df):
    key = df.keys()[-1]
    entropy = 0
    values = df[key].value_counts()
    total = len(df[key])
    for value in values:
        fraction = value/total
        entropy += -fraction*np.log2(fraction)
    return entropy

def find_entropy_attribute(df,attribute):
    key = df.keys()[-1]
    target_variables = df[key].unique()
    variables = df[attribute].unique()
    entropy2 = 0
    total = len(df)
    for variable in variables:
        entropy = 0
        den = len(df[attribute][df[attribute]==variable])
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[key] == target_variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/total
        entropy2 += -fraction2*entropy
    return abs(entropy2)


def find_winner(df):
    max_ig = 1e-10
    winner = None
    train_entropy = find_entropy(df)
    for key in df.keys()[:-1]:
        ig = train_entropy - find_entropy_attribute(df, key)
        if(ig > max_ig):
            max_ig = ig
            winner = key
    return winner

def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)

def buildTree(df, depth = -1):
    key = df.keys()[-1]

    if depth == 0:
        return df[key].value_counts().index[0]

    node = find_winner(df)
    if node is None:
        return df[key].value_counts().index[0]
    attValue = np.unique(df[node])

    tree = {}
    tree[node] = {}

    for value in attValue:        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable[key],return_counts=True)

        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable, depth-1)

    return tree

def predict_example(tree, example):
    while isinstance(tree, dict):
        attribute = list(tree.keys())[0]
        if example[attribute] not in tree[attribute].keys():
            return None
        tree = tree[attribute][example[attribute]]
    return tree

def predict(tree, df):
    predictions = []
    for index, row in df.iterrows():
        predictions.append(predict_example(tree, row))
    return predictions

def calc_accuracy(df, predictions):
    acc = sum(df.salary == predictions)/len(df)
    return acc

def get_best_tree(df,depth=-1):
    best_tree = None
    max_accuracy = 0.0
    avg_accuracy = 0.0
    best_seed = 0
    for x in range(10):
        seed = random.randint(0,100)
        train = df.sample(frac=0.8, random_state=seed)
        test = df.drop(train.index)
        train = train.reset_index(drop=True)
        test = test.reset_index(drop=True)
        t = buildTree(train, depth=depth)
        accuracy = calc_accuracy(test, predict(t, test))
        avg_accuracy += accuracy
        print('Accuracy : ', accuracy)
        if(max_accuracy < accuracy):
            best_seed = seed
            best_tree = t
            max_accuracy = accuracy
    
    avg_accuracy = avg_accuracy/1.0
    return best_tree, avg_accuracy, seed

def filter_df(df, attribute, values):
    df_split = {}
    for val in values:
        df_split[val] = df[df[attribute]==val].reset_index(drop=True)
    
    return df_split

def pruning_result(tree, df_train, df_val):

    leaf = df_train.salary.value_counts().index[0]
    errors_leaf = sum(df_val.salary != leaf)
    errors_decision_node = sum(df_val.salary != predict(tree, df_val))

    if errors_leaf <= errors_decision_node:
        return leaf
    else:
        return tree

def post_pruning(tree, df_train, df_val):
    
    attribute = list(tree.keys())[0]
    values = list(tree[attribute].keys())

    base_case = True
    for val in values:
        if isinstance(tree[attribute][val], dict):
            base_case = False
            break

    if base_case:
        return pruning_result(tree, df_train, df_val)
        
    # recursive part
    else:
        df_train_split = filter_df(df, attribute, values)
        df_val_split = filter_df(df_val, attribute, values)
        
        new_tree = {}
        
        for val in values:
            if isinstance(tree[attribute][val], dict):
                new_tree[val] = post_pruning(tree[attribute][val], df_train_split[val], df_val_split[val])
    
        new_tree = {attribute : new_tree}
        
        return pruning_result(new_tree, df_train, df_val)

In [ ]:
# finding best depth
depth_dict = []
for d in range(1, 11):
    t, acc, seed = get_best_tree(df, d)
    depth_dict.append([d, t, acc, seed])
    print(d, acc)

In [ ]:
# dividing data in training, validation and test set. 
df_train = df.sample(frac=0.6)
df_val = df.drop(df_train.index)
df_val = df_val.reset_index(drop=True)
df_test = df_val.sample(frac=0.5)
df_val = df_val.drop(df_test.index)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
# building the tree and pruning it
tree = buildTree(df_train, 3)
pruned_tree = post_pruning(tree, df_train, df_val)
print(calc_accuracy(df_val, predict(tree, df_val)), calc_accuracy(df_val, predict(pruned_tree, df_val)))

In [ ]:
pprint(pruned_tree)